### Purpose of Script
I'm not sure if huggingface keeps track of what we make the model output and if we might get flagged by their system for our use case. Just in case we should save the models locally and then load them from the disk instead of loading them from huggingface every time.

# This file is depreciated, use the following command instead

huggingface-cli download [MODEL_NAME_FROM_HUGGINGFACE. i.e. meta-llama/Llama-3.2-3B-Instruct] --local-dir [SAVE_PATH]


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
from dotenv import load_dotenv

/home/puschb/UVA/CBM/Information_Spread_Model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def save_model_locally(model_name, save_dir):
  load_dotenv()
  token = os.getenv("HF_ACCESS_TOKEN")
  # Download the model and tokenizer
  model = AutoModelForCausalLM.from_pretrained(
      model_name, 
      cache_dir=save_dir, 
      use_auth_token=token
  )
  tokenizer = AutoTokenizer.from_pretrained(
      model_name, 
      cache_dir=save_dir, 
      use_auth_token=token
  )


In [3]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
local_dir = "./Local Models/meta-llama_Llama-3.2-3B-Instruct"
save_model_locally(model_name,local_dir)

/home/puschb/UVA/CBM/Information_Spread_Model/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.81s/it]
/home/puschb/UVA/CBM/Information_Spread_Model/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
from transformers import pipeline
import torch


pipe = pipeline(
    "text-generation",
    model=local_dir,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.38it/s]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/puschb/UVA/CBM/Information_Spread_Model/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ValueError: Input length of input_ids is 52, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [22]:
import json
system_prompt_path = "system_prompt.txt"
with open(system_prompt_path, "r") as file:
    system_prompt = file.read()

with open('../Data/juanzy_politics_posts.json', 'r') as file:
    previous_comments = json.load(file)

previous_comments = '\n'.join([comment["body"] for comment in previous_comments])

post = "Trump, Who Was Charged with Mishandling Secrets, Poised to Get Classified Briefings Again"
comment_responding_to = '''Elon's untouchable *because* he's key to a major classified DoD initiative: https://www.reddit.com/r/EnoughMuskSpam/comments/1gbd7kg/elon_musk_is_insanely_dangerous/

Very good read.'''

system_prompt = system_prompt.format(previous_comments = previous_comments, post = post)
print(system_prompt)



You are a digital twin of a reddit user commenting on a political sub reddit. You will be given the all of your previous comments on 
posts in political subreddits. You will be given a comment that you have to respond to. You will respond to this comment in a manner that reflects your previous comments.

Previous comments on political subreddits: 
But he smoked weed on Rogan! He’s one of us!
You can’t blame us, Kamala wasn’t perfect! /s
They ask why people are anxious, nervous, or fearful then post some picture of a bullet with a caption like “the cure for liberalism”
It didn’t even take until election night for the hate to start coming out, and them playing victim.
Stubbornly stuck around in a one industry area, with an industry that died up 30 years ago?

It’s the immigrants fault!
Offered attractive one line “solutions” to complex problems
I know so many left wing people who took the "Both are bad" view. Usually the ones who stayed home were also white, protestant christian, and wel

In [23]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": comment_responding_to},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)

print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': 'You are a digital twin of a reddit user commenting on a political sub reddit. You will be given the all of your previous comments on \nposts in political subreddits. You will be given a comment that you have to respond to. You will respond to this comment in a manner that reflects your previous comments.\n\nPrevious comments on political subreddits: \nBut he smoked weed on Rogan! He’s one of us!\nYou can’t blame us, Kamala wasn’t perfect! /s\nThey ask why people are anxious, nervous, or fearful then post some picture of a bullet with a caption like “the cure for liberalism”\nIt didn’t even take until election night for the hate to start coming out, and them playing victim.\nStubbornly stuck around in a one industry area, with an industry that died up 30 years ago?\n\nIt’s the immigrants fault!\nOffered attractive one line “solutions” to complex problems\nI know so many left wing people who took the "Both are bad" view. Usually the ones who stayed home we

In [24]:
print(json.dumps(outputs[0]["generated_text"], indent = 2))


[
  {
    "role": "system",
    "content": "You are a digital twin of a reddit user commenting on a political sub reddit. You will be given the all of your previous comments on \nposts in political subreddits. You will be given a comment that you have to respond to. You will respond to this comment in a manner that reflects your previous comments.\n\nPrevious comments on political subreddits: \nBut he smoked weed on Rogan! He\u2019s one of us!\nYou can\u2019t blame us, Kamala wasn\u2019t perfect! /s\nThey ask why people are anxious, nervous, or fearful then post some picture of a bullet with a caption like \u201cthe cure for liberalism\u201d\nIt didn\u2019t even take until election night for the hate to start coming out, and them playing victim.\nStubbornly stuck around in a one industry area, with an industry that died up 30 years ago?\n\nIt\u2019s the immigrants fault!\nOffered attractive one line \u201csolutions\u201d to complex problems\nI know so many left wing people who took the